---


<h1 style="text-align:center"> 
Lung Cancer Classification using CT Data (Smoke Detector)
</h1><style>

--- 
## Libraries


In [1]:
import pandas as pd
import pylidc as pl
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#pylidc must be in lung-cancer-classification directory
print(pl.__file__)

/home/inesamorim46/3ano1sem/labiacd/proj_1/lung-cancer-classification/pylidc/__init__.py


---

## Importing XML files

In [3]:
plt.set_cmap('gray')

<Figure size 640x480 with 0 Axes>

In [4]:
#%pip install xlrd

In [5]:
nodule_counts_df = pd.read_excel('lidc-idri-nodule-counts-6-23-2015.xlsx')
nodule_counts_df
#maybe eliminar quarta e quinta coluna

,TCIA Patent ID,Total Number of Nodules*,Number of Nodules >=3mm**,Number of Nodules <3mm***,Unnamed: 4,Unnamed: 5
0,LIDC-IDRI-0001,4,1,3,NaN,NaN
1,LIDC-IDRI-0002,12,1,11,NaN,*total number of lesions that received either ...
2,LIDC-IDRI-0003,4,4,0,NaN,"**total number of lesions that received a ""nod..."
3,LIDC-IDRI-0004,4,1,3,NaN,"***total number of lesions that received a ""no..."
4,LIDC-IDRI-0005,9,3,6,NaN,NaN
...,...,...,...,...,...,...
1014,LIDC-IDRI-1009,2,1,1,NaN,NaN
1015,LIDC-IDRI-1010,10,1,9,NaN,NaN
1016,LIDC-IDRI-1011,4,4,0,NaN,NaN
1017,LIDC-IDRI-1012,1,1,0,NaN,NaN


In [6]:
pacient_data_df = pd.read_excel('tcia-diagnosis-data-2012-04-20.xls')
pacient_data_df 
#nem todos os pacientes têm historial registado (157 em 1010)

,TCIA Patient ID,"Diagnosis at the Patient Level\n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic\n",Diagnosis Method\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response,Primary tumor site for metastatic disease,"Nodule 1\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n",Nodule 1\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n,"Nodule 2\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n",Nodule 2\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n,"Nodule 3\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n",Nodule 3\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n,"Nodule 4\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n",Nodule 4\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n,"Nodule 5\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n",Nodule 5\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n
0,LIDC-IDRI-0068,3,4,Head & Neck Cancer,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LIDC-IDRI-0071,3,1,Head & Neck,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LIDC-IDRI-0072,2,4,Lung Cancer,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LIDC-IDRI-0088,3,0,Uterine Cancer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LIDC-IDRI-0090,2,3,NSCLC,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,LIDC-IDRI-0994,2,3,LUL Large cell CA,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,LIDC-IDRI-1002,2,2,non-small cell carcinoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,LIDC-IDRI-1004,2,3,LUL NSCLC,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,LIDC-IDRI-1010,0,0,lymphoma,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---


## Importing Annotations

In [48]:
from scipy.spatial.distance import cdist  # To calculate distances between nodules

In [79]:
def group_nodules_by_proximity(annotations  , distance_threshold):
    """ Group nodules manually based on spatial proximity using centroid distance. """
    
    centroids = []
    # Calculate centroids for each nodule
    for ann in annotations:
        centroid = np.array(ann[0].centroid)
        #print(f"Centroid: {centroid}")
        centroids.append(centroid)
    # Calculate distances between all pairs of centroids
    distances = cdist(centroids, centroids)
    
    # Group nodules based on a distance threshold
    groups = []
    visited = set()
    
    for i, centroid in enumerate(centroids):
        if i in visited:
            continue
        
        # Start a new group with the actual annotation, not the index
        group = [annotations[i][0]]
        visited.add(i)
        
        # Find all nodules within the distance threshold
        for j in range(len(centroids)):
            if i != j and distances[i, j] <= distance_threshold:
                group.append(annotations[j][0])  # Append the actual annotation instead of the index
                visited.add(j)
        
        groups.append(group)
    
    return groups

# Define a distance threshold (e.g., 10 mm)
DISTANCE_THRESHOLD = 10.0  # in millimeters

In [80]:
patients_nodules_features = []

for scan in pl.query(pl.Scan):
    patient_id = scan.patient_id

    if len(scan.annotations) == 0:
        print("Patient %s has no annotations" %(scan.patient_id))
        print("################################################")
        continue

    try:
        # Try to cluster annotations automatically
        nodules = scan.cluster_annotations()
        
        # Check if the clustering produced usable results (at least one group of nodules)
        if len(nodules) == 0:
            raise ValueError("Automatic clustering failed, falling back to manual grouping.")

    except Exception as e:
        # If automatic clustering fails, fallback to manual grouping
        print(f"Error with automatic clustering for patient {patient_id}: {e}")
        
        # Use manual grouping based on proximity
        print("Patient %s has %d annotations" %(scan.patient_id, len(scan.annotations)))
        annotations = [[annotation] for annotation in scan.annotations]  # Create individual nodule lists
        nodules = group_nodules_by_proximity(annotations, DISTANCE_THRESHOLD)
        print("################################################")

    for nodule in nodules:
        """nodule is a list of annotations (from the various radiologists)"""

        #the goal is to calculate the average of the annotations, so the end result is one annotation for each nodule
        texture_list = []
        spiculation_list = []
        lobulation_list = []
        margin_list = []
        sphericity_list = []
        calcification_list = []
        internal_structure_list = []
        subtlety_list = []
        malignancy_list = []

        for annotation in nodule:
            texture_list.append(annotation.texture)
            spiculation_list.append(annotation.spiculation)
            lobulation_list.append(annotation.lobulation)
            margin_list.append(annotation.margin)
            sphericity_list.append(annotation.sphericity)
            calcification_list.append(annotation.calcification)
            internal_structure_list.append(annotation.internalStructure)
            subtlety_list.append(annotation.subtlety)
            malignancy_list.append(annotation.malignancy)


        features = {
            "ID": nodule[0].id,
            "Scan_ID": nodule[0].scan_id,
            "Patient_ID": patient_id,
            "Texture": int(np.mean(texture_list)),
            "Spiculation": int(np.mean(spiculation_list)),
            "Lobulation": int(np.mean(lobulation_list)),
            "Margin": int(np.mean(margin_list)),
            "Sphericity": int(np.mean(sphericity_list)),
            "Calcification": int(np.mean(calcification_list)),
            "Internal Structure": int(np.mean(internal_structure_list)),
            "Subtlety": int(np.mean(subtlety_list)),
            "Malignancy": int(np.mean(malignancy_list)),
        }

        patients_nodules_features.append(features)

Patient LIDC-IDRI-0028 has no annotations
################################################
Patient LIDC-IDRI-0032 has no annotations
################################################
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Error with automatic clustering for patient LIDC-IDRI-0055: 
Patient LIDC-IDRI-0055 has 20 annotations
################################################
Patient LIDC-IDRI-0062 has no annotations
################################################
Patient LIDC-IDRI-0071 has no annotations
################################################
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Error with automatic clustering for patient LIDC-IDRI-0092: 
Patient LIDC-IDRI-0092 has 13 annotations
################################################
Patient LIDC-IDRI-0100 has no annotations
################################################
Failed to reduce all groups to 

In [81]:
df_annotations = pd.DataFrame(patients_nodules_features)
df_annotations

,ID,Scan_ID,Patient_ID,Texture,Spiculation,Lobulation,Margin,Sphericity,Calcification,Internal Structure,Subtlety,Malignancy
0,2,1,LIDC-IDRI-0078,4,2,2,3,4,6,1,4,3
1,1,1,LIDC-IDRI-0078,4,2,3,2,4,6,1,4,3
2,8,1,LIDC-IDRI-0078,5,1,1,5,5,5,1,4,1
3,3,1,LIDC-IDRI-0078,4,2,3,3,3,5,1,5,4
4,16,2,LIDC-IDRI-0069,4,4,4,4,4,6,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2656,6850,1016,LIDC-IDRI-0639,3,3,2,2,3,6,1,4,4
2657,6851,1016,LIDC-IDRI-0639,1,1,1,2,4,6,1,2,3
2658,6856,1017,LIDC-IDRI-0638,5,1,1,3,3,6,1,3,3
2659,6855,1017,LIDC-IDRI-0638,5,1,1,5,4,6,1,5,2


---

In [9]:
path_to_annotations = 'LIDC-XML-only'

In [ ]:
path = 'C:/Users/Daniela/Desktop/Fac/3ºano/Lab_IACD/Project1/manifest-1727162347556/LIDC-IDRI'
f = open('C:/Users/Daniela/pylidc.conf', 'w')
f.write(f'[dicom]\npath = {path}\n\n')
f.close()

In [53]:
pid = 'LIDC-IDRI-0001'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

In [ ]:
vol = scan.to_volume()
print(vol.shape) # (dim, dim, depth)

plt.figure(figsize=(5, 5))
plt.imshow(vol[:,:,0])
plt.show() # show the first-level slice

In [ ]:
#extract information from an XML file
def parse_xml(file):
    tree = ET.parse(file)
    root = tree.getroot()

    data = []
    for nodule in root

---

### Radiomics

In [52]:
import radiomics